### 1.Create Label

In [18]:
import pandas as pd
from pathlib import Path

current_dir = Path.cwd()
df = pd.read_csv(current_dir.parent/ "outputs" /"classification" / "merged_college_stats.csv")
df.head()

,SEASON,OVERALL_PICK,Totals_FG,Totals_FT,Totals_TRB,Totals_BLK,Totals_STL,Totals_TOV,Totals_PF,Shooting_FG%,MP
0,2000,1,221.0,141.0,300.0,107.0,43.0,56.0,71.0,0.568,909.0
1,2000,2,208.0,127.0,279.0,95.0,50.0,80.0,88.0,0.608,1013.0
2,2000,4,327.0,175.0,285.0,39.0,29.0,77.0,103.0,0.582,1243.0
3,2000,5,175.0,124.0,243.0,15.0,46.0,71.0,70.0,0.476,1058.0
4,2000,6,140.0,70.0,123.0,30.0,31.0,46.0,64.0,0.478,879.0


In [19]:
import numpy as np
import pandas as pd

# df is your merged dataset
# drafted = 1 if picked 1–60, undrafted (100) = 0
df["drafted"] = (df["OVERALL_PICK"] <= 60).astype(int)
print(df["drafted"].value_counts())


drafted
1    1210
0     552
Name: count, dtype: int64


### 2. Train Test Split

In [20]:
!pip install imbalanced-learn

You should consider upgrading via the '/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/bin/python3 -m pip install --upgrade pip' command.


In [21]:
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

# ============================
# 1. Train/Test Split by SEASON
# ============================
# Sort all seasons
all_seasons = sorted(df["SEASON"].unique())

# Number of seasons to use as test (20% of 25 = 5)
test_size = 5

# Select last 5 seasons
test_seasons = all_seasons[-test_size:]

# Remaining seasons are the training data
train_seasons = all_seasons[:-test_size]

print("Train seasons:", train_seasons)
print("Test seasons:", test_seasons)

# Create train_df and test_df
train_df = df[df["SEASON"].isin(train_seasons)].copy()
test_df  = df[df["SEASON"].isin(test_seasons)].copy()

print("Train seasons:", sorted(train_df["SEASON"].unique()))
print("Train size:", len(train_df), "Test size:", len(test_df))

# ============================
# 2. Features / Target
# ============================
target_col = "drafted"   # <-- 1 = drafted, 0 = undrafted

feature_cols = [
    "Totals_FG",
    "Totals_FT",
    "Totals_TRB",
    "Totals_STL",
    "Totals_BLK",
    "Totals_TOV",
    "Totals_PF",
    "Shooting_FG%",
    "MP",
]

X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_test  = test_df[feature_cols]
y_test  = test_df[target_col]

print("\nClass balance BEFORE UnderSampling (train):")
print(y_train.value_counts())

# ============================
# 3. UNDER-SAMPLE TRAIN
# ============================
rus_train = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus_train.fit_resample(X_train, y_train)

print("\nClass balance AFTER UnderSampling (train):")
print(pd.Series(y_train_resampled).value_counts())

# ============================
# 4. UNDER-SAMPLE TEST (because you asked)
# ============================

print("\nClass balance BEFORE UnderSampling (test):")
print(y_test.value_counts())

rus_test = RandomUnderSampler(random_state=42)
X_test_resampled, y_test_resampled = rus_test.fit_resample(X_test, y_test)

print("\nClass balance AFTER UnderSampling (test):")
print(pd.Series(y_test_resampled).value_counts())



Train seasons: [np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]
Test seasons: [np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
Train seasons: [np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]
Train size: 1430 Test size: 332

Class balance BEFORE UnderSampling (train):
drafted
1    938
0    492
Name: count, dtype: int64

Class balance AFTER UnderSampling (train):
drafted
0    492
1    492
Na

/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated i

In [22]:
X_train_resampled.head()

,Totals_FG,Totals_FT,Totals_TRB,Totals_STL,Totals_BLK,Totals_TOV,Totals_PF,Shooting_FG%,MP
35,190.0,151.0,161.0,49.0,8.0,97.0,83.0,0.438,1030.0
36,155.0,101.0,258.0,44.0,47.0,83.0,120.0,0.513,998.0
37,155.0,148.0,253.0,25.0,40.0,71.0,90.0,0.583,730.0
38,123.0,100.0,220.0,10.0,13.0,75.0,118.0,0.468,822.0
39,138.0,85.0,158.0,52.0,1.0,98.0,71.0,0.447,978.0


### 3. Train

In [23]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from scipy.stats import randint, uniform

# Base CV setup (same as before)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gb = GradientBoostingClassifier(random_state=42)

# Reasonable hyperparameter search space for GB
param_dist = {
    "n_estimators": randint(50, 301),          # number of trees
    "learning_rate": uniform(0.01, 0.19),      # 0.01–0.20
    "max_depth": randint(2, 5),                # shallow trees to reduce overfitting
    "min_samples_split": randint(2, 21),
    "min_samples_leaf": randint(1, 21),
    "subsample": uniform(0.6, 0.4),            # 0.6–1.0 (stochastic GB)
    "max_features": ["sqrt", "log2", None],
}

search = RandomizedSearchCV(
    estimator=gb,
    param_distributions=param_dist,
    n_iter=40,                 # increase to 80–100 later if you want
    scoring="roc_auc",         # focus on discrimination quality
    cv=cv,
    n_jobs=-1,
    random_state=42,
    return_train_score=True,
    verbose=1,
)

search.fit(X_train_resampled, y_train_resampled)

print("Best ROC-AUC (CV):", search.best_score_)
print("Best params:", search.best_params_)
best_model = search.best_estimator_


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best ROC-AUC (CV): 0.6593932873087394
Best params: {'learning_rate': np.float64(0.04128461209715893), 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': np.float64(0.7077649335194086)}


### 4.Evaluate

In [26]:
from sklearn.metrics import f1_score


# Fit on the full resampled train
best_model.fit(X_train_resampled, y_train_resampled)

# You can choose test_resampled or original test, depending on what you want
y_test_pred = best_model.predict(X_test_resampled)
y_test_proba = best_model.predict_proba(X_test_resampled)[:, 1]

print("Confusion matrix on TEST:")
print(confusion_matrix(y_test_resampled, y_test_pred))

print("\nClassification report on TEST:")
print(classification_report(y_test_resampled, y_test_pred, digits=3))

print("ROC-AUC on TEST:", roc_auc_score(y_test_resampled, y_test_proba))
test_f1 = f1_score(y_test_resampled, y_test_pred)
print("\nFINAL METRIC (F1-score used as accuracy):", round(test_f1, 3))


Confusion matrix on TEST:
[[36 24]
 [30 30]]

Classification report on TEST:
              precision    recall  f1-score   support

           0      0.545     0.600     0.571        60
           1      0.556     0.500     0.526        60

    accuracy                          0.550       120
   macro avg      0.551     0.550     0.549       120
weighted avg      0.551     0.550     0.549       120

ROC-AUC on TEST: 0.5243055555555556

FINAL METRIC (F1-score used as accuracy): 0.526
